# Visualize object detections

Draw bounding boxes on images to visualize object detection results.


## Problem

You've run object detection on images but need to visualize the results—see where objects were detected and verify the model's accuracy.

| Use case | Need |
|----------|------|
| Model evaluation | See what the model detected |
| Debugging | Verify detection accuracy |
| Reporting | Create annotated images for review |


## Solution

**What's in this recipe:**
- Run object detection with YOLOX
- Draw bounding boxes on images
- Color-code by object class

You create a pipeline that detects objects and then draws the results on the original image.


### Setup


In [ ]:
%pip install -qU pixeltable torch torchvision


In [ ]:
import pixeltable as pxt
from pixeltable.functions.yolox import yolox
from pixeltable.functions.vision import draw_bounding_boxes


In [ ]:
# Create a fresh directory
pxt.drop_dir('viz_demo', force=True)
pxt.create_dir('viz_demo')


### Create detection and visualization pipeline


In [ ]:
# Create table for images
images = pxt.create_table(
    'viz_demo.images',
    {'image': pxt.Image}
)


In [ ]:
# Step 1: Run object detection
images.add_computed_column(
    detections=yolox(images.image, model_id='yolox_m', threshold=0.5)
)


In [ ]:
# Step 2: Draw bounding boxes on the image
images.add_computed_column(
    annotated=draw_bounding_boxes(
        images.image,
        images.detections.bboxes,
        images.detections.labels,
        images.detections.scores
    )
)


### Detect and visualize


In [ ]:
# Insert sample images
base_url = 'https://raw.githubusercontent.com/pixeltable/pixeltable/main/docs/resources/images'

image_urls = [
    f'{base_url}/000000000036.jpg',  # cats
    f'{base_url}/000000000139.jpg',  # elephants
]

images.insert([{'image': url} for url in image_urls])


In [ ]:
# View original vs annotated images side by side
images.select(images.image, images.annotated).collect()


In [ ]:
# View detection details
for row in images.select(images.detections).collect():
    print("Detected objects:")
    for label, score in zip(row['detections']['labels'], row['detections']['scores']):
        print(f"  - {label}: {score:.2%}")


## Explanation

**Pipeline flow:**

```
Image → YOLOX detection → Bounding boxes + labels → draw_bounding_boxes → Annotated image
```

**Detection output format:**

The `yolox` function returns a dict with:
- `bboxes` - List of [x1, y1, x2, y2] coordinates
- `labels` - List of class names (e.g., "cat", "dog")
- `scores` - List of confidence scores (0-1)

**YOLOX model options:**

| Model | Size | Speed | Accuracy |
|-------|------|-------|----------|
| `yolox_nano` | 0.9M | Fastest | Lower |
| `yolox_tiny` | 5.1M | Fast | Good |
| `yolox_s` | 9.0M | Medium | Better |
| `yolox_m` | 25.3M | Slower | High |
| `yolox_l` | 54.2M | Slow | Higher |


## See also

- [Detect objects in images](https://docs.pixeltable.com/howto/cookbooks/images/img-detect-objects) - Object detection basics
- [Extract video frames](https://docs.pixeltable.com/howto/cookbooks/video/video-extract-frames) - Detect objects in video
